In [26]:
from lxml import html
import requests

In [3]:
page = requests.get('https://sis1.host.cs.st-andrews.ac.uk/GAP/MR2900886.html')
tree = html.fromstring(page.content)

### Manual selsection

Using the Chrome browser, I highlight the GAP citation text, right-click then select Inspect, then right-click on the text and select Copy Xpath - that gives me:

//*[@id="content"]/div[5]/div[4]/ol/li[10]/text()

The full Xpath for the GAP citation
/html/body/div[3]/div[4]/div[5]/div[4]/ol/li[10]/text()

In [3]:
gap = tree.xpath('//*[@id="content"]/div[5]/div[4]/ol/li[10]/text()')

In [4]:
print(gap)

['\n  Hoare, A.H.M., 1988. Pregroups and length functions. Math. Proc. Camb. Philos. Soc. 104 (1), 21–30. \n', '\n\n']


The above works, but I manually pointed it to the location of the citation.
I need to be able to find GAP citations in different result pages hence I will try the following:

### Xpath 'contains'

In [30]:
targets = tree.xpath('//li[contains(., "gap") or contains(.,"GAP")]')
for target in targets:
    print(target.text)


  The GAP Group, 


### With multiple URLs

In [27]:
url_lst = ['https://sis1.host.cs.st-andrews.ac.uk/GAP/MR4044696.html',
           'https://sis1.host.cs.st-andrews.ac.uk/GAP/MR2900886.html',
           'https://sis1.host.cs.st-andrews.ac.uk/GAP/MR3169623.html',
           'https://sis1.host.cs.st-andrews.ac.uk/GAP/MR4180136.html']


In [31]:
for page3 in url_lst:
    page3 = requests.get(page3)
    tree3 = html.fromstring(page3.content)
    targets3 = tree3.xpath('//li[contains(., "gap") or contains(.,"GAP")]')
    for target in targets3:
        print(target.text)


  The GAP Group, GAP â groups, algorithms and programming, version 4.10, Available from http://www.gap-system.org, 2018. 



  The GAP Group, 

  Distler, A., Mitchell, J. D. (2011). 

  The GAP Group, (2008). (http://www.gap-system.org). 

  Alonso, J., Brady, T., Cooper, D., Ferlini, V., Lustig, M., Mihalik, M., Shapiro, M., Short, H., 1991. Notes on word-hyperbolic groups. In: Ghys, E., Haefliger, A., Verjovsky, A. (Eds.), Proceedings of the Conference 

  The GAP Group, 2019. GAP â Groups, Algorithms, and Programming, Version 4.10.1; https://www.gap-system.org. 




It yields good results but the problem with extra tags inside < li > elements remains.

### Attempt to flatten the html using 'minidom'

In [46]:
from xml.dom import minidom
xmldoc = minidom.parse("MR4044696.html")
itemlist = xmldoc.getElementsByTagName('li')
print(len(itemlist))
for s in itemlist:
    print(s.attributes['li'].value)

FileNotFoundError: [Errno 2] No such file or directory: 'MR4044696.html'

Another attempt

In [43]:
import xml.etree.ElementTree as ET
root = ET.parse('MR4044696.html').getroot()



FileNotFoundError: [Errno 2] No such file or directory: 'MR4044696.html'